# Group beliefs

In [ ]:
import torch
from collections import deque
import os

import networkx as nx
import dgl

import matplotlib.pyplot as plt
import numpy as np

import polygraphs as pg
from polygraphs import graphs
from polygraphs import hyperparameters as hparams
from polygraphs import visualisations as viz
from polygraphs import ops
from polygraphs import init

%matplotlib inline

In [ ]:
# Create a PolyGraph configuration
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.001

params.network.kind = 'barabasialbert'
params.network.size = 128
params.network.barabasialbert.attachments = 4

# Enable logging; print progress every 100 steps
params.logging.enabled = True
params.logging.interval = 100

# Take snapshots
params.snapshots.enabled = True
params.snapshots.interval = 100

params.simulation.steps = 0
params.simulation.repeats = 1

# Store results in directory
params.simulation.results = "data/barabasialbert"

# Set seed
params.seed = 123456789

pg.random(params.seed)
_ = pg.simulate(params, op=ops.BalaGoyalOp)

## Analysis

In [ ]:
import h5py


# 1. Does a majority of nodes have a credence > 0.5?
# 2. Does a supermajority of nodes have a credence > 0.5?

# 3. Give each node its in degree number of votes, and let 
#    each node vote all its votes for B if its credence 
#    is > 0.5. Are a majority of votes for B?

# 4. Give each node its out degree number of votes, and let each node vote all its votes for B if its credence is > 0. 
# Are a majority of votes for B?

# 5. Take the average of the credences. Is this above 0.5?

# 6. Take the average weighted by… the local clustering co-efficient? (That doesn’t seem right, as it basically measures density. Maybe the size of the [in or out] neighbourhood of the node divided by the size of the network?) Is this > 0.5?

# 7. Geometric mean of credences (related to (5))?

# 8. ...


def sparsity(graph):
    """
    Returns sparsity level of given DGL graph.
    """
    # Remove self-loops
    g = dgl.remove_self_loop(graph)
    # Assumes an adjacency matrix of size N x N with M non-zero values
    return g.num_edges() / (g.num_nodes() ** 2)


def acc(graph):
    """
    Returns average clustering coefficient.
    """
    graphx = nx.DiGraph(dgl.to_networkx(graph))
    return nx.algorithms.cluster.average_clustering(graphx)


def apl(graph):
    """
    Returns average shortest path length.
    """
    graphx = nx.DiGraph(dgl.to_networkx(graph))
    return nx.average_shortest_path_length(graphx)


def majority(beliefs, threshold=0.5, weights=None):
    """
    Returns percentage of nodes that believe B is better.
    """
    if weights is None:
        weights = torch.ones(beliefs.shape)

    zeros = torch.zeros(beliefs.shape, dtype=weights.dtype)

    # Count (normalized) votes
    votes = torch.where(beliefs > threshold, weights, zeros)
    result = votes.sum() / weights.sum()
    return result.item()

def average_credence(beliefs):
    # print("Sum:", beliefs.sum())
    # print("# samples:", len(beliefs))
    return beliefs.sum() / len(beliefs)


def filterfn(edges):
    return torch.le(edges.src["beliefs"], 0.5)
   

def postprocess(directory, id):
    """
    Post-process graph snapshots
    """
    # Resulting hashtable
    ht = {}
    graphs, _ = dgl.load_graphs(os.path.join(directory, f"{id}.bin"))
    graph = graphs[0]
    # Remove self-loops
    graph = dgl.remove_self_loop(graph)
    # Weights are the in-degrees
    weights = graph.in_degrees()
    # Weights are the out-degrees
    # weights = graph.out_degrees()
    # No weights
    # weights = None
    fp = h5py.File(os.path.join(directory, f"{id}.hd5"), "r")
    _keys = [int(key) for key in fp.keys()]
    _keys = sorted(_keys)
    for key in _keys:
        beliefs = torch.tensor(fp[str(key)][:])
        ht[key] = majority(beliefs, threshold=0.7, weights=weights)
    return ht


ht = postprocess("data/complete", 1)

In [ ]:
plt.rc('font', **{'family':'sans-serif','sans-serif':['Arial'], 'size': 16})

# Configure the y-axis - find max value and use discrete steps
plt.ylim([0, 1])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.ylabel('Majority')
# Configure the x-axis
plt.xlim([0, max(ht.keys())])
plt.xlabel('Iterations')
# Create a bar chart
plt.plot(ht.keys(), 
         ht.values(),
         '--o',
         color='#B7BF35')